<a href="https://colab.research.google.com/github/mjavadzadeh11/BinanceAPI/blob/master/price_action_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas-ta

In [ ]:
import pandas_ta as ta
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import plotly.graph_objects as go
import plotly.express as px
import time
import json
import requests
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
def yf(autosave=False ,ticker='btc' , start_date=dt.date.today()- dt.timedelta(365) , end_date =dt.date.today() , interval = '1d' ):
  ''' set auto_save to True for auto_saving csv files'''
  
  As = 'On' if autosave else 'Off'

  print(f'auto saving has choosen {As}')
  interval = interval
  try:
    start_date = pd.to_datetime(start_date).date()
    end_date = pd.to_datetime(end_date).date()

    Ticker = ticker.upper() + '-USD'
    print(f'Process for  {Ticker} from {start_date} to {end_date} started.')
    period1 = int(time.mktime(start_date.timetuple()))
    period2 = int(time.mktime(end_date.timetuple()))

    query = f'https://query1.finance.yahoo.com/v7/finance/download/{Ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
    df=pd.read_csv(query , index_col = 'Date' ,parse_dates=True)
    if autosave:
      df.to_csv(f'{ticker}{end_date}.csv')
    return df , interval , ticker
  except:
    print("Input dates is incorrect! please insert the input dates in 'YYYYMD' format")

In [ ]:
df , interval  , ticker= yf()

auto saving has choosen Off
Process for  BTC-USD from 2021-02-19 to 2022-02-19 started.


'btc'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2021-02-09 to 2022-02-09
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       366 non-null    float64
 1   High       366 non-null    float64
 2   Low        366 non-null    float64
 3   Close      366 non-null    float64
 4   Adj Close  366 non-null    float64
 5   Volume     366 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 20.0 KB


In [ ]:
def Binance (symbol ='btc', interval='4h' , startTime= dt.datetime.today() - dt.timedelta(100) , endTime= dt.datetime.today()):
    ''' symbol="ETHUSDT or BTCUSDT , ..." , interval="1h,4h,8h,12h,1d", times = dt.datetime(2020,1,1) '''
    symbol = symbol.upper() + 'USDT'
    
    url = "https://api.binance.com/api/v3/klines"
    startTime= str(int(startTime.timestamp()*1000))
    endTime= str(int(endTime.timestamp()*1000))
    limit ="1000"
    req_p = {"symbol" : symbol , "interval": interval , 'startTime':startTime , 'endTime':endTime , 'limit':limit}
    response = pd.DataFrame(json.loads(requests.get(url , params = req_p).text))
    if len(response.index)==0 :
        return None
    response=response.iloc[:,0:6]
    response.columns=['Date' , 'Open' , 'High' , 'Low' , 'Close' , 'Volume' ]
    response['Date'] = [dt.datetime.fromtimestamp(x/1000) for x in response['Date']]
    response.set_index(['Date'] , inplace=True)
    return response , interval

In [ ]:
df , interval = Binance(symbol= 'dot')
df=df.applymap(lambda x: float(x))

In [ ]:
df

In [ ]:
def diff(node , list):
  eliminate=[]
  for arg in list:
    if (node-arg)/node <0.04  and (node-arg)/node >0 and node!=arg:
      eliminate.append(arg)
    elif (node-arg)/node <0  and (node-arg)/node >-0.04 and node!=arg:
      eliminate.append(arg)
    else:
      pass
  return eliminate

In [ ]:
def point_adder(list):
  
  list.append([dt.datetime.today() , list[0][1]])
  return list

In [ ]:
def closest(list, K ): 
  output=[]
  lst= [SR[1] for SR in list]
  cv= lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]
  for i in list:
    if i[1]==cv:
      output.append([i[0] , cv])
  return output

In [ ]:
def Static(df=df , pivot=14):

  if isinstance(df.index ,pd.core.indexes.datetimes.DatetimeIndex) :
    df.reset_index(inplace=True)
 
  #finding peaks:
  Strong_Resistance = []
  Weak_Resistance = []
  Strong_Support = []
  Weak_Support=[]

  for i in range (pivot ,df.shape[0]-pivot):
    # For strong resistance:
    lower_values = [df.loc[i , 'High'] for i in range(i-(pivot-1), i)]
    higher_values = [df.loc[i,'High'] for i in range(i+1, i+pivot)]

    if df.loc[i, 'High'] > max(lower_values) and df.loc[i,'High'] > max(higher_values) and len(diff(df.loc[i, 'High'],[SR[1] for SR in Strong_Resistance]))==0  :   # and len(diff(df.loc[i, 'High'],[SR[1] for SR in Strong_Resistance]))==0
      
      
      Strong_Resistance.append([df.loc[i,'Date'],df.loc[i, 'High']])

    #For weak resistance:

    lows=[df.loc[i , 'High'] for i in range(i-(int(pivot/2)-1) ,i)]
    highs=[df.loc[i,'High'] for i in range(i+1, i+int(pivot/2))]
    if df.loc [i ,'High'] > max(lows) and df.loc [i,'High'] >max(highs):
      Weak_Resistance.append([df.loc[i,'Date'],df.loc [i ,'High']])

  #Finging Vallys:
  for i in range ( pivot , df.shape[0]-pivot):
    #Storng Support:
    lower_values= [df.loc[i,'Low'] for i in range(i-(pivot-1), i) ]
    higher_values = [df.loc[i,'Low'] for i in range(i+1, i+pivot )] 
    if df.loc[i, 'Low'] < min(lower_values) and df.loc[i,'Low'] < min (higher_values) and len(diff(df.loc[i, 'Low'],[SR[1] for SR in Strong_Support]))==0:
      Strong_Support.append([df.loc[i,'Date'],df.loc[i, 'Low']])

        #Weak Support:
    lower_values= [df.loc[i,'Low'] for i in range(i-(int(pivot/2)-1), i) ]
    higher_values = [df.loc[i,'Low'] for i in range(i+1, i+int(pivot/2) )] 
    if df.loc[i, 'Low'] < min(lower_values) and df.loc[i,'Low'] < min (higher_values):
      Weak_Support.append([df.loc[i,'Date'],df.loc[i, 'Low']])

    #Return index
  df.set_index(['Date'] , inplace=True)
  #adding point and find closest Static line from last price:
  #Strong_Resistance = point_adder(closest( Strong_Resistance , df.loc[df.index[-1] , 'Close'] ))
  #Strong_Support = point_adder(closest( Strong_Support , df.loc[df.index[-1] , 'Close'] ))


  return  Strong_Resistance , Strong_Support 

In [ ]:
Strong_Resistance ,Strong_Support  =Static()

In [ ]:
Strong_Resistance

[[Timestamp('2021-03-13 00:00:00'), 61683.863281],
 [Timestamp('2021-04-14 00:00:00'), 64863.097656],
 [Timestamp('2021-06-15 00:00:00'), 41295.269531],
 [Timestamp('2021-09-07 00:00:00'), 52853.765625],
 [Timestamp('2021-11-10 00:00:00'), 68789.625]]

In [ ]:
output=point_adder(closest( Strong_Resistance , df.loc[df.index[-1] , 'Close'] ))
output1=point_adder(closest( Strong_Support , df.loc[df.index[-1] , 'Close'] ))

In [ ]:
output

[[Timestamp('2021-06-15 00:00:00'), 41295.269531],
 [datetime.datetime(2022, 2, 19, 11, 43, 46, 154861), 41295.269531]]

In [ ]:
def point_append(data , interval , df=df):
  ''' point adder to trend line'''


  slope = (data.Values[1]-data.Values[0]) / (data.Index[1] - data.Index[0])
  coef = data.Values[0] - slope * data.Index[0]
  
  distance = date_sub(data,interval)      #(df.Index[1] - df.Index[0])*0.7 + df.Index[1] #(dt.datetime.now()-df.loc[1,'Date Index']  ).days   + df.loc[1,'Index']
  new_y= (slope * distance) + (coef)
  date_index =df.loc[df.index[-1] , 'Date']  #df.loc[df.index[-1] , 'Date']  #df.loc[1,'Date Index'] + dt.timedelta(int((df.Index[1] - df.Index[0]) ))
  

  list=[distance, date_index,new_y ]
  data.loc[2] = list

  return data 

In [ ]:
def date_sub(df, interval):
  import math 

  if interval[-1] =='h': #hourly distance base on interval
    interval = int(interval[0:-1])
    total_time=str(dt.datetime.now() -df.loc[1,'Date Index']).split(' ')
    day = int(total_time[0])
    hour =math.floor( int( total_time[2].split(':')[0]) ) #math.ceil
    #mintue = int(total_time[2].split(':')[1])/60
    
    #epoch base on interval
    epoch = day *24 / interval + hour/interval

   

    return  epoch + df['Index'][1]

  elif interval[-1] =='d': #daily distance
    return (dt.datetime.now()-df.loc[1,'Date Index']  ).days   + df.loc[1,'Index'] 
    
  elif interval[-1] =='w': # weekly distance
    
    return math.ceil((dt.datetime.now()-df.loc[1,'Date Index']  ).days/ 7 )+ df.loc[1,'Index']

  elif interval[-1] =='M':
    return math.ceil((dt.datetime.now()-df.loc[1,'Date Index']  ).days/ 30 )+ df.loc[1,'Index']


In [ ]:
def trend_line(df,pivot):
  ''' pivot: number of candel in pivot , n: how many trend line '''

  if isinstance(df.index ,pd.core.indexes.datetimes.DatetimeIndex):
    df.reset_index(inplace=True)
  highs =[]
  lows=[]

  for i in range(pivot , df.shape[0]-pivot):
    lower = [df.loc[i,'High'] for i in range(i-(pivot-1), i)]
    higher = [df.loc[i,'High'] for i in range(i+1, i+pivot)]
    if df.loc[i,'High'] > max(lower) and df.loc[i,'High'] > max(higher):
      highs.append([i,df.loc[i,'Date'] , df.loc[i,'High']])
  highs = pd.DataFrame(highs)
  highs.columns = ['Index' , 'Date Index' , 'Values']
  
 
  highs=highs.tail(2)
  highs.reset_index(drop=True , inplace=True)
  
  highs = point_append(highs , interval)

  for i in range(pivot , df.shape[0]-pivot):
    lower = [df.loc[i,'Low'] for i in range(i-(pivot-1), i)]
    higher = [df.loc[i,'Low'] for i in range(i+1, i+pivot)]
    if df.loc[i,'Low'] < min(lower) and df.loc[i,'Low'] < min(higher):
      lows.append([i, df.loc[i,'Date'], df.loc[i,'Low']] )

  lows = pd.DataFrame(lows)
  lows.columns = ['Index' , 'Date Index' , 'Values']
  lows=lows.tail(2)
  lows.reset_index(drop=True , inplace=True)
  lows=point_append(lows , interval)
  #returning index
  
  df.set_index(['Date'] , inplace=True)


  
  return highs , lows


In [ ]:
highs , lows = trend_line(df,10)

In [ ]:
#=====================================================volume_based supoort and resistance =====================================================#

In [ ]:
def vol_profile(df , num_of_candles=list):
  '''calculate volume of trades based on price, 
  num_of_candles is a list which we can pass to function to calculate volumne based on them. for instance [7,30].'''

  if isinstance(df.index ,pd.core.indexes.datetimes.DatetimeIndex) :
    df.reset_index(inplace=True)
    
  volume = {}
  for i in num_of_candles:

    temp = df.iloc[-i:,:]

    temp.reset_index(drop=True , inplace=True)

    max_price = max(temp['High'])
    min_price=min(temp['Low'])
    
    zones = np.arange(min_price,max_price,( max_price-min_price) /24)
    
    volume_profile = {}

    for j in zones:

      for m in range(temp.shape[0]):

        if j > temp.loc[m,'Low'] and j< temp.loc[m,'High']:
          volume_profile.setdefault( j , temp.loc[m,'Volume'])
          volume_profile[j]+= temp.loc[m,'Volume']


    max_key = max(volume_profile, key=volume_profile.get)

    line= [[dt.datetime.today() - dt.timedelta(i) , max_key]]
    line= point_adder(line)
    volume.setdefault( i , line)

  return volume



In [ ]:
volume = vol_profile(df , [30,50])

In [ ]:
fig = go.Figure()
fig.add_trace(
        go.Candlestick(
        x=df.index,
        low=df['Low'],
        high=df['High'],
        close=df['Close'],
        open=df['Open'],
        increasing_line_color='green',
        decreasing_line_color='red'
    ))

x=[i[0] for i in output]
y=[i[1] for i in output]
fig.add_trace(

  go.Scatter(x=x ,y=y ,line_color='#0000ff' ,name = 'Static Line')

  )
x=[i[0] for i in output1]
y=[i[1] for i in output1]

fig.add_trace(

  go.Scatter(x=x ,y=y ,line_color='#FF3C33' , name = 'Static Line' )

  )




fig.add_trace(
    
    go.Scatter(x=highs['Date Index'] , y=highs['Values']  , name='Trend Line')
)

for i in volume.keys():
  x=[volume[i][j][0] for j in [0,1]]
  y=[volume[i][j][1] for j in [0,1]]

  fig.add_trace(
      go.Scatter(x=x , y=y  , name = f'Highest Tranded Zone in past {i} candle(s)')
  )



fig.update_layout(
    width=1350,
    height=1000 , 
    title = f'{ticker.upper()} Timeframe:{interval}'
)

fig.show()

In [ ]:
#===================================================== Divergence **  Calculation =====================================================#

In [ ]:
ta.indicators()

AttributeError: ignored